# EDGAR - Reading Information Tables in Text Format - Advanced Text Mining (25 marks)

#### So far, we have collected CIKs for each of the Mutual Funds, then looked up the links of all the 13F HRs and the Information Tables, and identified them either text tables or xml tables. In class, we  obtained the information from the xml tables.  In this assignment we will obtain information from the text file.  These are not as nicely structured as the xml output.   In a csv (HW_Mutual_Fund_Info_Table_Link.csv) you will find a few of these links.  Your goal for this part of the homework is to obtain the link to the text files from the attached file  (HW_Mutual_Fund_Info_Table_Link.csv).  Then you will write a code that will go to the links of all the linked text files, and extract some columns.  Do not use Beutiful Soup for this assignment.   We provide some initial code to guide your initial steps:

#### First we have to collect the links of the text Info Tables in lists:

In [6]:
import urllib

text_link = []
text_Name = []
text_date = []

input_file = open('HW_Mutual_Fund_Info_Table_Link.csv', 'r')

rows = input_file.readlines()
input_file.close()

# Your Code on Enumerating the Lists.  The result should be three lists, text_link, text_Name,
#and text_date.  Each should have length 122.
for row in range(len(rows)):
    new = rows[row].split(",")
    if new[4] == "text":
        text_link.append(new[3])
        text_Name.append(new[1])
        text_date.append(new[2])
len(text_link),len(text_Name),len(text_date)

(122, 122, 122)

#### Keep only the links that correspond to a date after 2010 (Don't inlude 2010, start at 2011).  Hint: you can use the datetime library.

In [7]:
from datetime import datetime

#Use the following list to store the filtered values.
filtered_dates = []
filtered_name = []
filtered_link = []

#Enter code here to to keep only the dates corresponding to after 2010.  
for row in range(len(text_date)):
    if datetime.strptime(text_date[row],'%Y-%m-%d') >= datetime(2011,1,1):
        filtered_dates.append(text_date[row])
        filtered_name.append(text_Name[row]) 
        filtered_link.append(text_link[row])

filtered_dates,filtered_name,filtered_link

(['2013-05-06',
  '2013-05-01',
  '2013-02-08',
  '2012-11-07',
  '2012-08-09',
  '2012-05-15',
  '2012-02-02',
  '2011-02-14'],
 ['Adirondack_Funds',
  'ADVANCE_CAPITAL_I_INC',
  'ADVANCE_CAPITAL_I_INC',
  'ADVANCE_CAPITAL_I_INC',
  'ADVANCE_CAPITAL_I_INC',
  'ADVANCE_CAPITAL_I_INC',
  'ADVANCE_CAPITAL_I_INC',
  'Global_X_Funds'],
 ['/Archives/edgar/data/1311981/000116204413000513/0001162044-13-000513.txt',
  '/Archives/edgar/data/813470/000081347013000006/0000813470-13-000006.txt',
  '/Archives/edgar/data/813470/000081347013000001/0000813470-13-000001.txt',
  '/Archives/edgar/data/813470/000081347012000023/0000813470-12-000023.txt',
  '/Archives/edgar/data/813470/000081347012000019/0000813470-12-000019.txt',
  '/Archives/edgar/data/813470/000081347012000014/0000813470-12-000014.txt',
  '/Archives/edgar/data/813470/000081347012000003/0000813470-12-000003.txt',
  '/Archives/edgar/data/1432353/000114420411008428/0001144204-11-008428.txt'])

#### Your filtered list should now have 8 elements.  These represent 3 mutual funds.   The first one has CIK=1311981 (from the link you can find this at data/1311981 ).  The second was has CIK 813470.  The third one has CIK 1432353. 

['https://www.sec.gov/Archives/edgar/data/1311981/000116204413000513/0001162044-13-000513.txt ',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347013000006/0000813470-13-000006.txt ',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347013000001/0000813470-13-000001.txt ',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000023/0000813470-12-000023.txt ',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000019/0000813470-12-000019.txt ',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000014/0000813470-12-000014.txt ',
 'https://www.sec.gov/Archives/edgar/data/813470/000081347012000003/0000813470-12-000003.txt ',
 'https://www.sec.gov/Archives/edgar/data/1432353/000114420411008428/0001144204-11-008428.txt ']

#### Next, for each text link, extract the name of issuer, CUSIP, and the Quantity of shares.  You will also want to keep track of the mutual fund name as well as the filing report date.  

#### Your output file should have 5 columns.  The first is the issue date of the form which can be found in the filtered_date list (this will be repeated for the same form).  The second is the mutual fund name which can be found in the filtered_name list (this will be repeated).  The third, fourth and fifith are the name of issuer, CUSIP, and shares respectively.  Make sure to account for the fact that while some of the text files have the same formatting, others do not.  This means you will have to look through them to make sure your code works for the each text file.

In [8]:
issue_date = []
mutual_fund_name = [] 
name_of_issuer = [] 
CUSIP = [] # CUSIP number
shares = [] # No. of Shares of the company in the Mutual Fund

for link in range(len(filtered_link)):
    url = 'https://www.sec.gov'+filtered_link[link]
    html = urllib.request.urlopen(url).read().decode('utf-8')
    
    while html.find("<CAPTION>") != -1:
        index = html.find("<CAPTION>") 
        html2 = html[index+10:]
        index_Start = html2.find("<")
        index_End = html2.find("</TABLE>")
        
        if html2.find("GRAND")!= -1:
            index_End = html2.find("GRAND")
            html2 = html2[index_Start:index_End]
        else:
            html2 = html2[index_Start:index_End]  #going closer to the results table
   
        findIndex = html2.split("\n")
        #Find Issuer Name Index for each line
        NameIndex_Start = findIndex[0].find("<")
        NameIndex_End = findIndex[0][NameIndex_Start+1:].find("<")+1
        #Find CUSIP Index for each line
        CUSIP_Start = findIndex[0].find("<",NameIndex_End+1)
        CUSIP_End = findIndex[0].find("<",CUSIP_Start+1)
        #Fine Share Index for each line
        Share_Start = findIndex[0].find("<",CUSIP_End+1)
        Share_End = findIndex[0].find("<",Share_Start+1)
    
        # for file1 and file7, we have different startlines to get data.
        if link == 0:
            startline = 3
        elif link == 6:
            startline = 4
        else:
            startline = 1
    
        for row in range(startline,len(findIndex)):
            # remove blank items, if the issuer name is seperated in 2 lines, add the second line to the previous line.
            if findIndex[row][NameIndex_Start:NameIndex_End].strip() != '':
                if findIndex[row][CUSIP_Start:CUSIP_End]!='':
                    issue_date.append(filtered_dates[link])
                    mutual_fund_name.append(filtered_name[link])
                    name_of_issuer.append(findIndex[row][NameIndex_Start:NameIndex_End].strip())
                    CUSIP.append(findIndex[row][CUSIP_Start:CUSIP_End].strip())
                    shares.append(findIndex[row][Share_Start:Share_End].strip())
                else:
                    name_of_issuer[-1] = name_of_issuer[-1]+findIndex[row][NameIndex_Start:NameIndex_End].strip()
        html = html[index_End:]


#### Finally, you write down the lists in the following file:

In [9]:
# Your code goes here
Output_File = open("Mutual_Fund_Info_Table.txt", 'w')
Output_File.write("Issue_date"+'\t'+"Name"+'\t'+"Name_of_Issuer"+'\t'+"CUSIP"+"\t"+"Shares"+"\n")
for x in range(len(issue_date)):
    Output_File.write(issue_date[x] + '\t' + mutual_fund_name[x] + '\t' + name_of_issuer[x] + '\t' + CUSIP[x] + '\t' + shares[x] + '\n')

Output_File.close()
